 Terminal 1: Start Zookeeper

In [ ]:
cd C:\kafka
.\bin\windows\zookeeper-server-start.bat .\config\zookeeper.properties


 Terminal 2: Start Kafka Broker

In [ ]:
cd C:\kafka
.\bin\windows\kafka-server-start.bat .\config\server.properties


 Terminal 3: Create Topic

In [ ]:
cd C:\kafka
.\bin\windows\kafka-topics.bat --create --topic market-data-topic --bootstrap-server localhost:9092 --partitions 1 --replication-factor 1


Final Step: Run Your MT5 to Kafka Python Script Again


In [ ]:
producer = KafkaProducer(
    bootstrap_servers='localhost:9093',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)


Python script that connects to MT5, fetches live tick data, and sends it to Kafka (topic: market-data-topic) using kafka-python.

✅ ✅ ✅

In [ ]:
import MetaTrader5 as mt5
from kafka import KafkaProducer
import json
import time
from datetime import datetime

# Initialize MT5 connection
def connect_to_mt5():
    account = *****
    password = "****"
    server = "*****"

    if not mt5.initialize():
        print("❌ MT5 initialization failed")
        quit()

    if not mt5.login(account, password=password, server=server):
        print("❌ MT5 login failed:", mt5.last_error())
        mt5.shutdown()
        quit()

    print("✅ Connected to MT5")

# Kafka Producer with safe serializer
def json_serializer(v):
    def convert(o):
        if isinstance(o, (int, float, str, bool)):
            return o
        elif isinstance(o, (datetime,)):
            return o.isoformat()
        elif hasattr(o, "item"):  # For NumPy types like np.uint64, np.float32
            return o.item()
        else:
            return str(o)

    return json.dumps(v, default=convert).encode("utf-8")

producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=json_serializer
)

# Get real-time tick data and push to Kafka
def stream_mt5_to_kafka(symbol="EURUSD", topic="market-data-topic"):
    connect_to_mt5()

    print(f"📡 Streaming real-time data for {symbol} → Kafka topic: '{topic}'")

    while True:
        ticks = mt5.copy_ticks_from(symbol, datetime.now(), 1, mt5.COPY_TICKS_ALL)
        for tick in ticks:
            data = {
                "timestamp": datetime.fromtimestamp(tick['time']).isoformat(),
                "instrument": symbol,
                "price": float(tick['ask']),
                "volume": float(tick['volume'])
            }
            print("📤 Sending:", data)
            producer.send(topic, value=data)

        time.sleep(1)

if __name__ == "__main__":
    try:
        stream_mt5_to_kafka(symbol="EURUSD", topic="market-data-topic")
    except KeyboardInterrupt:
        print("\n⛔ Interrupted by user")
    finally:
        mt5.shutdown()
        print("🛑 MT5 connection closed")


NoBrokersAvailable: NoBrokersAvailable